In [1]:
import pandas as pd
import numpy as np
import os # To check if the file exists
from scipy.stats import norm # For Parametric VaR and CVaR
from scipy.stats import chi2 # For Kupiec's backtesting test

# --- Configuration ---
# Define the path to your Excel file. Make sure this file is in the same directory
# as your Python script, or provide the full path to it.
excel_file_path = 'Historical_Portfolio.xlsx'
sheet_name = 'portfolio' # Changed as per your update

# Define the confidence level for VaR and ES calculation
confidence_level = 0.99

# For Monte Carlo Simulation
num_simulations = 10000 # Number of simulated portfolio returns

# --- Data Loading and Preparation ---

# Check if the Excel file exists before attempting to load
if not os.path.exists(excel_file_path):
    print(f"Error: The file '{excel_file_path}' was not found.")
    print("Please ensure the Excel file is in the correct directory.")
else:
    try:
        # Load the historical stock price data from the specified sheet
        # We'll set the 'Date' column as the index for time-series analysis
        df_prices = pd.read_excel(excel_file_path, sheet_name=sheet_name, index_col='Date')

        print("--- Original Data Head ---")
        print(df_prices.head())
        print("\n--- Data Info ---")
        df_prices.info()
        print("\n--- Data Description ---")
        print(df_prices.describe())

        # Calculate daily returns for each stock
        df_returns = df_prices.pct_change()

        # Drop the first row which will contain NaN values due to pct_change()
        df_returns = df_returns.dropna()

        print("\n--- Daily Returns Head ---")
        print(df_returns.head())
        print("\n--- Daily Returns Info ---")
        df_returns.info()

        # --- Portfolio Construction (Assuming Equal Weighting for now) ---
        num_assets = df_returns.shape[1]
        portfolio_weights = np.array([1/num_assets] * num_assets)
        print(f"\nPortfolio Weights (Equal): {portfolio_weights}")

        # Calculate daily portfolio returns
        portfolio_returns = df_returns.dot(portfolio_weights)

        print("\n--- Portfolio Returns Head ---")
        print(portfolio_returns.head())
        
        # Calculate mean and standard deviation of portfolio returns for Parametric and Monte Carlo VaR
        portfolio_mean_return = portfolio_returns.mean()
        portfolio_std_dev = portfolio_returns.std()

        print(f"Mean Portfolio Return: {portfolio_mean_return:.4f}")
        print(f"Standard Deviation of Portfolio Returns: {portfolio_std_dev:.4f}")

        # --- Historical Value-at-Risk (VaR) Calculation ---
        # Sort the portfolio returns in ascending order (from worst loss to best gain)
        sorted_returns = portfolio_returns.sort_values(ascending=True)

        # Calculate the percentile for VaR (e.g., 0.01 for 99% confidence)
        var_percentile = 1 - confidence_level

        # Calculate Historical VaR
        historical_var = sorted_returns.quantile(var_percentile)

        print(f"\n--- Market Risk Metrics ---")
        print(f"Historical VaR ({confidence_level*100:.0f}% Confidence Level): {historical_var:.4f} (or {historical_var*100:.2f}%)")

        # --- Historical Conditional Value-at-Risk (CVaR) / Expected Shortfall (ES) Calculation ---
        cvar_returns_historical = sorted_returns[sorted_returns <= historical_var]
        historical_cvar = cvar_returns_historical.mean()
        print(f"Historical CVaR / Expected Shortfall ({confidence_level*100:.0f}% Confidence Level): {historical_cvar:.4f} (or {historical_cvar*100:.2f}%)")

        # --- Parametric (Delta-Normal) VaR Calculation ---
        z_score = norm.ppf(var_percentile)
        parametric_var = portfolio_mean_return + (z_score * portfolio_std_dev)
        print(f"\nParametric (Delta-Normal) VaR ({confidence_level*100:.0f}% Confidence Level): {parametric_var:.4f} (or {parametric_var*100:.2f}%)")

        # --- Parametric CVaR / Expected Shortfall Calculation ---
        pdf_at_z_score = norm.pdf(z_score)
        parametric_cvar = portfolio_mean_return - (portfolio_std_dev * pdf_at_z_score) / (1 - var_percentile)
        print(f"Parametric CVaR / Expected Shortfall ({confidence_level*100:.0f}% Confidence Level): {parametric_cvar:.4f} (or {parametric_cvar*100:.2f}%)")


        # --- Monte Carlo VaR Calculation ---
        simulated_returns = np.random.normal(loc=portfolio_mean_return,
                                             scale=portfolio_std_dev,
                                             size=num_simulations)
        sorted_simulated_returns = np.sort(simulated_returns)
        monte_carlo_var = np.percentile(sorted_simulated_returns, var_percentile * 100)
        print(f"\nMonte Carlo VaR ({confidence_level*100:.0f}% Confidence Level, {num_simulations} simulations): {monte_carlo_var:.4f} (or {monte_carlo_var*100:.2f}%)")

        # --- Monte Carlo CVaR / Expected Shortfall Calculation ---
        cvar_returns_mc = sorted_simulated_returns[sorted_simulated_returns <= monte_carlo_var]
        monte_carlo_cvar = cvar_returns_mc.mean()
        print(f"Monte Carlo CVaR / Expected Shortfall ({confidence_level*100:.0f}% Confidence Level, {num_simulations} simulations): {monte_carlo_cvar:.4f} (or {monte_carlo_cvar*100:.2f}%)")


        # --- VaR Model Backtesting (Kupiec's Proportion of Failures Test) ---
        print("\n--- VaR Model Backtesting (Kupiec's Proportion of Failures Test) ---")

        # For demonstration, we'll backtest the Historical VaR against the full historical returns.
        # In a real-world scenario, VaR would be re-estimated daily/periodically using a rolling window.
        
        # Number of observations (days)
        T = len(portfolio_returns)
        
        # Count the number of actual losses that exceeded the Historical VaR (exceptions)
        # Remember VaR is typically a negative number (a loss), so 'exceeded' means more negative.
        exceptions = portfolio_returns[portfolio_returns < historical_var].count()
        
        # Expected number of exceptions
        expected_exceptions = T * (1 - confidence_level)
        
        # Proportion of failures (actual exceptions / total observations)
        p_hat = exceptions / T
        
        # Null hypothesis: The model is accurate (observed proportion of failures equals expected proportion)
        # H0: p_hat = (1 - confidence_level)
        # Alternative hypothesis: The model is inaccurate
        
        # Kupiec's POF test statistic (Likelihood Ratio Test)
        # L_LR_POF = -2 * ln( ( (1-p)^(T-N) * p^N ) / ( (1-p_hat)^(T-N) * p_hat^N ) )
        # where p = 1 - confidence_level, N = exceptions, T = total observations
        
        # Handle cases where p_hat or (1-p_hat) might be zero to avoid log(0)
        # Also handle cases where expected_exceptions might be zero if T is very small and confidence_level is high
        
        # To avoid log(0) or division by zero, use small epsilon for probabilities if they are exactly 0 or 1
        epsilon = 1e-10

        p = 1 - confidence_level
        
        # Numerator of the likelihood ratio (under H0)
        L_num = (p**exceptions) * ((1-p)**(T-exceptions))
        
        # Denominator of the likelihood ratio (under H1, using observed p_hat)
        # Ensure p_hat is not exactly 0 or 1 for log calculation
        p_hat_safe = np.clip(p_hat, epsilon, 1 - epsilon)
        L_den = (p_hat_safe**exceptions) * ((1-p_hat_safe)**(T-exceptions))

        # Avoid log of zero by adding a small epsilon if L_num or L_den is zero
        LR_POF = -2 * np.log(np.clip(L_num / L_den, epsilon, 1/epsilon))
        
        # Critical value for chi-squared distribution with 1 degree of freedom at 95% confidence
        alpha_significance = 0.05 # 5% significance level
        critical_value = chi2.ppf(1 - alpha_significance, df=1) # 1 degree of freedom for POF test

        print(f"Total Observations (T): {T}")
        print(f"Observed VaR Exceptions (N): {exceptions}")
        print(f"Expected VaR Exceptions: {expected_exceptions:.2f}")
        print(f"Observed Proportion of Failures (p_hat): {p_hat:.4f}")
        print(f"Kupiec's LR_POF Test Statistic: {LR_POF:.4f}")
        print(f"Critical Value (Chi-squared, 1 df, {alpha_significance*100:.0f}% significance): {critical_value:.4f}")

        if LR_POF > critical_value:
            print("Conclusion: REJECT the VaR model (too many or too few exceptions).")
            print("The observed number of exceptions is statistically significantly different from the expected number.")
        else:
            print("Conclusion: DO NOT REJECT the VaR model.")
            print("The observed number of exceptions is not statistically significantly different from the expected number.")

        # --- CVaR Model Backtesting (Average Exceedance Test) ---
        print("\n--- CVaR Model Backtesting (Average Exceedance Test) ---")

        # Get the actual losses on VaR exception days
        # We need to filter the portfolio returns that are less than the historical VaR
        # These are the "exceedances" or "violations"
        var_violations = portfolio_returns[portfolio_returns < historical_var]

        if len(var_violations) > 0:
            # Calculate the average of these actual losses (exceedances)
            average_actual_exceedance_loss = var_violations.mean()
            
            print(f"Number of VaR violations: {len(var_violations)}")
            print(f"Average Actual Loss on VaR Violation Days: {average_actual_exceedance_loss:.4f} (or {average_actual_exceedance_loss*100:.2f}%)")
            print(f"Predicted Historical CVaR ({confidence_level*100:.0f}% Confidence Level): {historical_cvar:.4f} (or {historical_cvar*100:.2f}%)")

            # Compare the average actual exceedance loss to the predicted Historical CVaR
            # A simple comparison: ideally, average_actual_exceedance_loss should be close to historical_cvar.
            # If average_actual_exceedance_loss is significantly worse (more negative) than historical_cvar,
            # the CVaR model might be underestimating tail risk.
            # If average_actual_exceedance_loss is significantly better (less negative) than historical_cvar,
            # the CVaR model might be overestimating tail risk.

            # You can define a tolerance level for "close" or use more advanced statistical tests
            # (e.g., conditional coverage tests like Christoffersen's or tests based on tick losses)
            # For this basic implementation, we'll just show the comparison.
            
            # Simple qualitative assessment:
            if average_actual_exceedance_loss < historical_cvar:
                print("Qualitative CVaR Backtest: Model might be UNDERESTIMATING tail losses (Actual average loss worse than predicted CVaR).")
            elif average_actual_exceedance_loss > historical_cvar:
                print("Qualitative CVaR Backtest: Model might be OVERESTIMATING tail losses (Actual average loss better than predicted CVaR).")
            else:
                print("Qualitative CVaR Backtest: Model's predicted CVaR is consistent with average actual losses on violation days.")
        else:
            print("No VaR violations observed, cannot perform CVaR backtesting (Average Exceedance Test).")


        # --- Scenario Analysis and Stress Testing ---
        print("\n--- Scenario Analysis & Stress Testing ---")

        # Get the last available prices as the starting point for stress scenarios
        # Assuming the last row of df_prices contains the most recent prices
        current_prices = df_prices.iloc[-1]
        print("\nCurrent Asset Prices (as of last data point):")
        print(current_prices)

        # Assuming an initial investment of $1,000,000 total value for demonstration.
        # Distribute this value equally among assets based on their last price to find 'shares_held'.
        initial_portfolio_value = 1_000_000
        initial_investment_per_asset = initial_portfolio_value / num_assets
        shares_held = initial_investment_per_asset / current_prices
        
        current_portfolio_value = (current_prices * shares_held).sum()
        print(f"Current Portfolio Value (based on assumed ${initial_portfolio_value:,.2f} initial investment): ${current_portfolio_value:,.2f}")


        # Define Stress Scenarios as percentage shocks
        # Added historical scenarios based on approximate Indian market declines
        scenarios = {
            'Market Downturn (All stocks -15%)': {
                'Cipla_close': -0.15,
                'TATACON_close': -0.15,
                'BEL_close': -0.15
            },
            'Tech Sector Shock (BEL -25%)': { # Assuming BEL is tech-related for this example
                'Cipla_close': -0.05,  # Moderate impact
                'TATACON_close': -0.05, # Moderate impact
                'BEL_close': -0.25    # Severe impact
            },
            'Pharma Sell-off (Cipla -20%)': { # Assuming Cipla is pharma
                'Cipla_close': -0.20,
                'TATACON_close': -0.05,
                'BEL_close': -0.05
            },
            '2008 Global Financial Crisis (Approx. -35% across board)': {
                'Cipla_close': -0.35,
                'TATACON_close': -0.35,
                'BEL_close': -0.35
            },
            'COVID-19 Market Decline (Approx. -30% across board)': {
                'Cipla_close': -0.30,
                'TATACON_close': -0.30,
                'BEL_close': -0.30
            }
        }

        stress_test_results = {}

        for scenario_name, shocks in scenarios.items():
            # Apply shocks to current prices
            stressed_prices = current_prices.copy()
            for asset, shock_percentage in shocks.items():
                if asset in stressed_prices.index: # Ensure the asset exists in our current_prices
                    stressed_prices[asset] *= (1 + shock_percentage)
            
            # Calculate portfolio value under stress
            stressed_portfolio_value = (stressed_prices * shares_held).sum()
            
            # Calculate the loss under stress
            loss_under_stress = current_portfolio_value - stressed_portfolio_value
            
            stress_test_results[scenario_name] = {
                'Stressed Portfolio Value': stressed_portfolio_value,
                'Loss Under Stress': loss_under_stress,
                'Percentage Loss': (loss_under_stress / current_portfolio_value) * 100
            }

        print("\n--- Stress Test Results ---")
        for scenario_name, results in stress_test_results.items():
            print(f"\nScenario: {scenario_name}")
            print(f"  Stressed Portfolio Value: ${results['Stressed Portfolio Value']:,.2f}")
            print(f"  Loss Under Stress: ${results['Loss Under Stress']:,.2f}")
            print(f"  Percentage Loss: {results['Percentage Loss']:.2f}%")

        # Interpretation of Stress Testing:
        # Stress testing provides insight into how the portfolio would perform under
        # specific, predefined adverse market conditions. Unlike VaR/CVaR, which are statistical
        # measures based on historical data or simulations, stress tests use
        # hypothetical, extreme scenarios to assess worst-case outcomes.
        # This is crucial for capital planning and regulatory compliance (e.g., Basel, FRTB).

        # --- FRTB (Fundamental Review of the Trading Book) Considerations ---
        print("\n--- FRTB (Fundamental Review of the Trading Book) Considerations ---")
        print("FRTB is a key regulatory framework for market risk capital requirements.")
        print("Our project's components contribute to FRTB compliance as follows:")

        print("\n1. Internal Model Approach (IMA) vs. Standardised Approach (SA):")
        print("   - IMA requires banks to demonstrate robust risk models (VaR, ES, Stress Testing).")
        print("   - Our Historical, Parametric, and Monte Carlo VaR/ES calculations are foundational for IMA.")
        print("   - SA involves prescribed risk sensitivities and risk factors, which would require a different calculation methodology.")

        print("\n2. Expected Shortfall (ES):")
        print("   - FRTB mandates ES (at 97.5% confidence) as the primary risk measure for IMA, replacing VaR.")
        print("   - Our project explicitly calculates Historical, Parametric, and Monte Carlo CVaR (ES), aligning with this requirement.")

        print("\n3. Backtesting:")
        print("   - FRTB requires rigorous backtesting of ES models (though the specific tests are still evolving).")
        print("   - Our Kupiec's POF test for VaR is a basic form of backtesting. For FRTB, more advanced tests (e.g., conditional coverage, magnitude tests for ES) would be necessary.")
        print("   - Backtesting results determine whether a bank's internal model remains approved by regulators.")

        print("\n4. Stress Testing:")
        print("   - FRTB requires comprehensive stress testing to capture risks not fully covered by ES.")
        print("   - Our scenario analysis and stress testing framework directly supports this, allowing for the assessment of losses under extreme market movements.")

        # --- FRTB: Non-Modellable Risk Factors (NMRF) ---
        print("\n--- FRTB: Non-Modellable Risk Factors (NMRF) ---")
        print("FRTB introduces capital charges for Non-Modellable Risk Factors (NMRFs).")
        print("These are risk factors that cannot be adequately modeled due to insufficient observable data.")
        print("For this project, we'll conceptually illustrate how NMRF capital add-ons would be calculated.")
        
        # Example NMRFs and their hypothetical capital add-ons (as a percentage of exposure or fixed amount)
        # In a real scenario, these would be derived from regulatory guidelines or internal assessments.
        nmrf_factors = {
            'Illiquid_Bond_Spread': 0.02,  # 2% of exposure for illiquid bond spread risk
            'Exotic_FX_Volatility': 0.015, # 1.5% of exposure for exotic FX vol risk
            # Add more hypothetical NMRFs relevant to your portfolio if applicable
        }

        # For demonstration, let's assume a hypothetical exposure for each NMRF
        # In a real bank, this would be linked to specific instruments in the trading book.
        hypothetical_exposure_nmrf = {
            'Illiquid_Bond_Spread': 500_000,  # $500,000 exposure
            'Exotic_FX_Volatility': 200_000,  # $200,000 exposure
        }

        total_nmrf_capital_add_on = 0
        print("\nCalculating Capital Add-ons for Hypothetical NMRFs:")
        for factor, charge_rate in nmrf_factors.items():
            exposure = hypothetical_exposure_nmrf.get(factor, 0)
            capital_add_on = exposure * charge_rate
            total_nmrf_capital_add_on += capital_add_on
            print(f"  - {factor}: Exposure=${exposure:,.2f}, Charge Rate={charge_rate*100:.2f}%, Add-on=${capital_add_on:,.2f}")

        print(f"\nTotal Estimated NMRF Capital Add-on: ${total_nmrf_capital_add_on:,.2f}")
        print("Note: This is a simplified illustration. Actual NMRF calculation involves complex methodologies and regulatory approval.")

        # --- FRTB: Default Risk Charge (DRC) ---
        print("\n--- FRTB: Default Risk Charge (DRC) ---")
        print("FRTB introduces a separate capital charge for default risk in the trading book.")
        print("This covers idiosyncratic and jump-to-default risks, distinct from market price risk.")
        print("For this project, we'll conceptually illustrate a simplified DRC calculation.")

        # Hypothetical trading book instruments with credit risk
        # In a real scenario, these would be actual bonds, loans, or derivatives with counterparty risk.
        trading_book_instruments = [
            {'name': 'Corporate Bond A', 'EAD': 1_000_000, 'PD': 0.01, 'LGD': 0.40, 'correlation_factor': 0.5},
            {'name': 'Corporate Bond B', 'EAD': 750_000,  'PD': 0.02, 'LGD': 0.35, 'correlation_factor': 0.5},
            # Add more instruments as needed
        ]

        total_drc = 0
        print("\nCalculating Simplified DRC for Hypothetical Trading Book Instruments:")
        for instrument in trading_book_instruments:
            # Simplified DRC calculation (ignoring netting, hedging, jump-to-default correlation for simplicity)
            # This is a highly simplified Expected Loss component for demonstration.
            # Actual DRC is much more involved, often using a Merton-like model or credit spread approach.
            idiosyncratic_drc = instrument['EAD'] * instrument['PD'] * instrument['LGD']
            
            # For systemic risk (correlation factor), a more complex formula would be used.
            # Here, we just add a conceptual component based on a correlation factor.
            # This is NOT the full FRTB DRC formula but a conceptual illustration.
            systemic_drc_component = instrument['EAD'] * instrument['PD'] * instrument['LGD'] * instrument['correlation_factor'] # Highly simplified
            
            # For demonstration, let's just use the idiosyncratic component as a proxy for 'risk'
            # The actual FRTB DRC is a more complex measure of unexpected loss from default.
            instrument_drc = idiosyncratic_drc # Using only idiosyncratic part for simplicity
            total_drc += instrument_drc
            print(f"  - {instrument['name']}: EAD=${instrument['EAD']:,.2f}, PD={instrument['PD']*100:.2f}%, LGD={instrument['LGD']*100:.2f}%, Simplified DRC=${instrument_drc:,.2f}")

        print(f"\nTotal Estimated Simplified DRC: ${total_drc:,.2f}")
        print("Note: This is a highly simplified illustration of DRC. Actual FRTB DRC is complex, involving specific formulas for idiosyncratic and systematic risk components, and is calibrated to a 99.9% confidence level over a 1-year horizon.")


        # --- P&L Attribution (PLA) ---
        print("\n--- FRTB: P&L Attribution (PLA) ---")
        print("FRTB requires a P&L Attribution test to ensure that the risk factors used in the internal risk model adequately explain the daily P&L of the trading desk.")
        print("This involves comparing 'Hypothetical P&L' (from risk factor changes) with 'Actual P&L'.")
        print("This project does not currently implement PLA. It would require:")
        print("  - Daily P&L data for the portfolio.")
        print("  - A mapping of P&L to specific risk factors.")
        print("  - Calculation of P&L explained by the model's risk factors vs. unexplained P&L.")
        print("  - Statistical tests (e.g., Spearman's rank correlation, Kolmogorov-Smirnov test) to compare the distributions of hypothetical and actual P&L.")


    except Exception as e:
        print(f"An error occurred during data processing: {e}")

--- Original Data Head ---
            Cipla_close   TATACON_close   BEL_close 
Date                                                
2025-06-02        1470.2          1120.4      387.50
2025-05-30        1465.7          1106.3      384.60
2025-05-29        1476.9          1109.8      386.80
2025-05-28        1468.5          1121.4      390.45
2025-05-27        1480.5          1138.4      385.40

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 248 entries, 2025-06-02 to 2024-06-03
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Cipla_close     248 non-null    float64
 1   TATACON_close   248 non-null    float64
 2   BEL_close       248 non-null    float64
dtypes: float64(3)
memory usage: 7.8 KB

--- Data Description ---
       Cipla_close   TATACON_close   BEL_close 
count    248.000000      248.000000  248.000000
mean    1515.797581     1069.459677  296.545927
std       65.277478      